In [1]:
import argparse
import numpy as np
import tensorflow as tf

from scada_forecast.preprocess import read_humidity, read_temperature, read_scada
from scada_forecast.preprocess import get_lag_features, merge_dataframes, add_calendar_features, prepare_data

from scada_forecast.model import train_model, inference

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
def hourahead(task, 
              scada_path='data/scada/Dữ liệu SCADA Phụ tải 26.08.2020.xlsx', 
              scada_sheet='SCADA_QGia',
              use_humidity=True,
              humidity_path='data/scada/DoAm.xlsx', 
              use_temperature=True, 
              temperature_path='data/NhietDoQuaKhu.xlsx', 
              use_lunar=True, use_holiday=False, 
              n_val=100 * 24 * 12, n_test=100 * 24 * 12, # train only
              forecast_horizon=4, # inference only
              ckpt_path='tmp/ckpt'):
    """
    Dự báo phụ tải SCADA trước 1 giờ
    Args:
        task: str - 'train'-> train mô hình, 'inference' -> dự báo
        scada_path: str - đường dẫn file excel scada
        scada_sheet: str - chọn sheet cho file excel scada
        use_humidity: bool - có sử dụng dữ liệu độ ẩm hay không
        humidity_path: str - đường dẫn file excel độ ẩm
        use_temperature: bool- có sử dụng dữ liệu nhiệt độ hay không
        temperature_path: str - đường dẫn file excel nhiệt độ
        use_lunar: bool - có sử dụng các feature âm lịch hay không
        use_holiday: bool - có sử dụng các feature ngày nghỉ lễ hay không
        n_val: int - số điểm dữ liệu dùng cho tập validation (default 100 ngày) (dùng cho task='train')
        n_test: int - số điểm dữ liệu dùng cho tập test (default 100 ngày cuối) (dùng cho task='train')
        forecast_horizon: int - số điểm dự báo (bắt đầu từ thời điểm sau 1h đổ về trước) 
                                (default chỉ dự báo tại 1 thời điểm sau 1h) (dùng cho task='inference')
        ckpt_path: str - đường dẫn lưu model
    Returns:
        pandas.DataFrame lưu kết quả dự báo
    """
    df_humidity = None
    df_temperature = None
    df_scada = None
    hour_steps = int(60 / 5)
    input_width = hour_steps
    categorical_unique_values_dict = {
        'Month': list(range(1, 12 + 1)),
        'DayOfYear': list(range(1, 365 + 1)),
        'DayOfWeek': list(range(7)),
        'Hour': list(range(24)),
        'LunarMonth': list(range(1, 12 + 1)),
        'LunarDayOfMonth': list(range(1, 30 + 1)),
        'HolidayCat': list(range(12))
    }
    
    # TIEN XU LY
    print('Doc du lieu scada...')
    df_scada = read_scada(scada_path, sheet_name=scada_sheet)
    if task == 'train':
        df_scada = get_lag_features(df_scada, hour_steps, clip_df=False)
    if task == 'inference':
        df_scada = get_lag_features(df_scada, hour_steps, clip_df=True, 
                                    input_width=input_width, forecast_horizon=forecast_horizon)
    
    if use_humidity:
        print('Doc du lieu do am...')
        df_humidity = read_humidity(humidity_path)
    if use_temperature:
        print('Doc du lieu nhiet do...')
        df_temperature = read_temperature(temperature_path)
        
    df = merge_dataframes(df_scada, df_temp=df_temperature, df_humidity=df_humidity)
    print('Them cac feature thoi gian (gio, thu, ngay, thang)...')
    df = add_calendar_features(df, use_lunar=use_lunar, use_holiday=use_holiday)
    
    print('Chuan bi du lieu cho mo hinh...')
    df, features_dict, categorical_cols, numeric_cols, target_col = prepare_data(df, 
                                                                                 train_flag=(task=='train'),
                                                                                 use_temp=use_temperature, 
                                                                                 use_humidity=use_humidity, 
                                                                                 use_lunar=use_lunar,
                                                                                 use_holiday=use_holiday)
    
    dtype_dict = {col: df[col].dtype for col in categorical_cols + numeric_cols}
    chosen_features = (features_dict['calendar_features'] + features_dict['lag_features'] + 
                       features_dict['temperature_features'] + features_dict['humidity_features'])
    
    # TRAIN MO HINH
    if task == 'train':
        print('Training...')
        train_model(df, input_width, n_val, n_test, hour_steps,
                    dtype_dict, categorical_unique_values_dict, 
                    chosen_features, categorical_cols, 
                    numeric_cols, target_col, ckpt_path)
        print('Hoan thanh.')
        
    # DU BAO
    if task == 'inference':
        print('Inference...')
        forecast_df = inference(df, input_width, ckpt_path,
                                dtype_dict, categorical_unique_values_dict, 
                                chosen_features, categorical_cols, numeric_cols)
        print(forecast_df)
        return forecast_df

In [3]:
forecast_df = hourahead(task='inference',
                        scada_path='data/scada/Dữ liệu SCADA Phụ tải 26.08.2020.xlsx', 
                        scada_sheet='SCADA_QGia',
                        use_humidity=True,
                        humidity_path='data/scada/DoAm.xlsx', 
                        use_temperature=False,
                        use_lunar=True, use_holiday=False,
                        forecast_horizon=4,
                        ckpt_path='tmp/ckpt')

Doc du lieu scada...
Doc du lieu do am...
Them cac feature thoi gian (gio, thu, ngay, thang)...


F:\Anaconda3\envs\new_sklearn\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel



Chuan bi du lieu cho mo hinh...
Inference...
                         Forecast
Date                             
2020-08-26 00:40:00  29083.449219
2020-08-26 00:45:00  28938.640625
2020-08-26 00:50:00  28899.009766
2020-08-26 00:55:00  28735.078125


In [4]:
forecast_df

,Forecast
Date,
2020-08-26 00:40:00,29083.449219
2020-08-26 00:45:00,28938.640625
2020-08-26 00:50:00,28899.009766
2020-08-26 00:55:00,28735.078125
